In [ ]:
!python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py train /u/horlavanasta/MasterProject//DataAndExperiments/Experiments/Experiments-1.5T-3T/NNs_Bayesian/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-WeightedCrossEntropy_augmTrue --n_splits 1 --split 0  --batch_size 5               

In [ ]:
def available_scans(source_dir, ext=".pt"):
    import os
    subject_ids=[]
    for (dirpath, dirnames, filenames) in os.walk(source_dir):
        for f in filenames:
            if f.endswith(ext):
                subject_ids.append(f)
    return subject_ids

In [10]:
with open("scripts/submit_experiments.sh", "r") as f:
    lines=f.readlines()
for i in range(1, len(lines)):
    lines[i]="sbatch run_experiment.sh "+lines[i]
with open("scripts/submit_experiments.sh", "w") as f:
    for line in lines:
        f.write(line)
        
    
#     f.write("\nsbatch run_experiment.sh " + args.output_dir)

In [5]:
f="/u/horlavanasta/MasterProject//DataAndExperiments/Experiments_3-fold/Experiments-3T/NNs_Bayesian/Conv5_FC3/subject_model-Conv5_FC3_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmTrue_20210528_134641"

CAPS_DIR="$HOME/MasterProject/DataAndExperiments/Data/CAPS"
NN_FOLDER="NNs_Bayesian"
TSV_PATH="$HOME/MasterProject/DataAndExperiments/Experiments_3-fold/Experiments-3T/labels/test"
OUTPUT_DIR="$HOME/MasterProject//DataAndExperiments/Experiments_3-fold/Experiments-3T/NNs_Bayesian/Conv5_FC3/"

POSTFIX="test_3T"
BAYESIAN=True
NBR_BAYESIAN_ITER=10

!python /u/horlavanasta/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py classify $CAPS_DIR $TSV_PATH $f $POSTFIX --bayesian $BAYESIAN --nbr_bayesian_iter $NBR_BAYESIAN_ITER --selection_metrics balanced_accuracy loss last_checkpoint


TEST_POSTFIX="test_1.5T"
TEST_TSV_PATH="$HOME/MasterProject/DataAndExperiments/Experiments_3-fold/Experiments-1.5T/labels/"
!python /u/horlavanasta/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py classify $CAPS_DIR $TEST_TSV_PATH $f $TEST_POSTFIX --bayesian $BAYESIAN --nbr_bayesian_iter $NBR_BAYESIAN_ITER --selection_metrics balanced_accuracy loss last_checkpoint --baseline False

In [6]:

folders=[f.path for f in os.scandir("/u/horlavanasta/MasterProject/DataAndExperiments/Data/CAPS_pt/subjects/")]
print(len(folders))

983


In [3]:
import pathlib
import pandas as pd
import os
import json
from clinicaaddl.visualize.plot import plot_generic

folders = []
MS_main_list = ['1.5T', '3T', "1.5T-3T"]
MS_list_dict = {'1.5T':['1.5T', '3T'], "3T": ['3T', '1.5T'], "1.5T-3T": ["1.5T-3T"]}
home_folder='/u/horlavanasta/MasterProject/'

isBayesian=True
for MS in MS_main_list[:]:
    print("MS ____________________________________________________________________________________________")
    model_types = [ "ResNet18", "SEResNet18", "ResNet18Expanded", "SEResNet18Expanded", "Conv5_FC3" ]
    MS_list = MS_list_dict[MS]

    results_folder_general =os.path.join(home_folder, 'Code/ClinicaTools/AD-DL/results/', "Experiments_Bayesian" if isBayesian else "Experiments", 'Experiments-' + MS)
    model_dir_general = os.path.join(home_folder,"DataAndExperiments/Experiments/Experiments-" + MS, "NNs_Bayesian" if isBayesian else "NNs")

    for network in model_types[:]:
        model_dir = os.path.join(model_dir_general, network)
        # output_dir = pathlib.Path(output_dir)
        modelPatter = "subject_model*"
        folders = [f for f in pathlib.Path(model_dir).glob(modelPatter)]

        for f in folders[:]:
            
            results_dir=os.path.join(results_folder_general,network)
#             !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py bayesian $f stat
            !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir uncertainty_catplot --catplot_type stripplot --uncertainty_metric "total_variance"
            !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir uncertainty_catplot --catplot_type violinplot --uncertainty_metric "total_variance"
#             !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir uncertainty_catplot --catplot_type swarmplot --uncertainty_metric "entropy"
#             !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir uncertainty_catplot --catplot_type violinplot --uncertainty_metric "entropy"

#             !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir uncertainty_distribution
            !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir barplots_loss --save_best True --path_to_best $results_dir

MS ____________________________________________________________________________________________
MS ____________________________________________________________________________________________
MS ____________________________________________________________________________________________


In [2]:
import seaborn as sns
print(sns.__version__)

0.10.0


In [ ]:
import os
folders=[]
for f in os.scandir("/u/horlavanasta/MasterProject/DataAndExperiments/Data/CAPS_pt/subjects/"):
    if 'sub-' in f.path:
        pass
    folders.append(f.path)
print(len(folders))

In [ ]:
for f in folders:
    !echo $f
    !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py classify ~/MasterProject/ADNI_data/CAPSPreprocessedT1linear ~/MasterProject/ADNI_data/DataPrep/labels_new/test $f test